## Database Implementation and Data Querying

In [90]:
import sqlite3
import pandas as pd

db_path = "./Database/flight_data.db"
flights_csv  = "./Dataset/flights.csv"     
airports_csv = "./Dataset/airports.csv"
airlines_csv = "./Dataset/airlines.csv"

In [91]:
airports_df  = pd.read_csv(airports_csv)
airlines_df  = pd.read_csv(airlines_csv)

airports_df = airports_df[["IATA_CODE", "AIRPORT", "CITY", "A_STATE",
                           "COUNTRY", "LATITUDE", "LONGITUDE"]].drop_duplicates()
airlines_df = airlines_df[["IATA_CODE", "AIRLINE"]].drop_duplicates()


In [92]:
flights_df = pd.read_csv(flights_csv)

flights_df = flights_df[["A_YEAR", "A_MONTH","A_DAY","DAY_OF_WEEK","AIRLINE","FLIGHT_NUMBER",
                         "TAIL_NUMBER", "ORIGIN_AIRPORT",
                         "DESTINATION_AIRPORT","SCHEDULED_DEPARTURE", "DEPARTURE_TIME", "DEPARTURE_DELAY", "TAXI_OUT",
                            "WHEELS_OFF", "SCHEDULED_TIME", "ELAPSED_TIME", "AIR_TIME", "DISTANCE", "WHEELS_ON", "TAXI_IN",
                         "SCHEDULED_ARRIVAL", "ARRIVAL_TIME", "ARRIVAL_DELAY", "DIVERTED", "CANCELLED",
                         "CANCELLATION_REASON", "AIR_SYSTEM_DELAY", "SECURITY_DELAY", "AIRLINE_DELAY",
                         "LATE_AIRCRAFT_DELAY", "WEATHER_DELAY"]].drop_duplicates()
flights_df.insert(0, "flight_id",  range(1, len(flights_df)+1))

In [93]:
con = sqlite3.connect(db_path)
con.execute("PRAGMA foreign_keys = ON;")
cur = con.cursor()

for tbl in ("flights", "airports", "airlines"):
    cur.executescript(f"DROP TABLE IF EXISTS {tbl};")

cur.executescript("""
CREATE TABLE airports(
    IATA_CODE TEXT PRIMARY KEY,
    AIRPORT TEXT,
    CITY TEXT,
    A_STATE TEXT,
    COUNTRY TEXT,
    LATITUDE REAL,
    LONGITUDE REAL
);
""")

cur.executescript("""
CREATE TABLE airlines(
    IATA_CODE    TEXT PRIMARY KEY,
    AIRLINE      TEXT
);
""")

cur.executescript("""
CREATE TABLE flights(
    flight_id INT PRIMARY KEY,
    A_YEAR INT,
    A_MONTH INT,
    A_DAY INT,
    DAY_OF_WEEK INT,
    AIRLINE TEXT,
    FLIGHT_NUMBER INT,
    TAIL_NUMBER TEXT,
    ORIGIN_AIRPORT TEXT,
    DESTINATION_AIRPORT TEXT,
    SCHEDULED_DEPARTURE INT,
    DEPARTURE_TIME INT,
    DEPARTURE_DELAY INT,
    TAXI_OUT INT,
    WHEELS_OFF INT,
    SCHEDULED_TIME INT,
    ELAPSED_TIME INT,
    AIR_TIME INT,
    DISTANCE INT,
    WHEELS_ON INT,
    TAXI_IN INT,
    SCHEDULED_ARRIVAL INT,
    ARRIVAL_TIME INT,
    ARRIVAL_DELAY INT,
    DIVERTED INT,
    CANCELLED INT,
    CANCELLATION_REASON TEXT,
    AIR_SYSTEM_DELAY INT,
    SECURITY_DELAY INT,
    AIRLINE_DELAY INT,
    LATE_AIRCRAFT_DELAY INT,
    WEATHER_DELAY INT,
                  
    FOREIGN KEY (ORIGIN_AIRPORT) REFERENCES airports(IATA_CODE),
    FOREIGN KEY (DESTINATION_AIRPORT) REFERENCES airports(IATA_CODE),
    FOREIGN KEY (AIRLINE) REFERENCES airlines(IATA_CODE)
);
""")
con.commit()


In [94]:
airports_df.to_sql("airports", con, if_exists="append", index=False)
airlines_df.to_sql("airlines", con, if_exists="append", index=False)
flights_df.to_sql("flights",  con, if_exists="append", index=False)

con.commit()
con.close()
print("SQLite database built at", db_path)


SQLite database built at ./Database/flight_data.db


In [95]:
import sqlite3, pandas as pd
from tabulate import tabulate
def run_sql(
    sql: str,
    params: tuple | None = None,
    head_rows: int = 6,       
    tail_rows: int = 3,        
    tablefmt: str = "psql",      
    return_df: bool = False,
):
    with sqlite3.connect(db_path) as con:
        con.execute("PRAGMA foreign_keys = ON;")
        df = pd.read_sql_query(sql, con, params=params)

    n = len(df)
    if n == 0:
        print("(0 rows)")
        return pd.DataFrame() if return_df else None

    if n <= 20:
        short = df
    else:
        ellipsis_row = pd.DataFrame(
            {c: "…" for c in df.columns}, index=["…"]
        )
        short = pd.concat([df.head(head_rows), ellipsis_row, df.tail(tail_rows)])
    in_nb = False
    try:
        from IPython import get_ipython
        in_nb = "IPKernelApp" in get_ipython().config
    except Exception:
        pass

    if in_nb:
        from IPython.display import display, HTML
        display(short)                      
        display(HTML(f"<em>{n:,} rows total</em>"))
    else:
        print(tabulate(short, headers="keys", tablefmt=tablefmt, showindex=False))
        print(f"\n[{n:,} rows total]")

    return df if return_df else None


In [96]:
run_sql("""
SELECT *
FROM airports
""")


,IATA_CODE,AIRPORT,CITY,A_STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.6819
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
5,ACK,Nantucket Memorial Airport,Nantucket,MA,USA,41.25305,-70.06018
…,…,…,…,…,…,…,…
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023
321,YUM,Yuma International Airport,Yuma,AZ,USA,32.65658,-114.60597


In [97]:
run_sql("""
SELECT *
FROM airlines
""")


,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [98]:
run_sql("""
SELECT *
FROM flights
""")

,flight_id,A_YEAR,A_MONTH,A_DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,1,2015,1,1,4,AS,98,N407AS,ANC,SEA,...,408.0,-22.0,0,0,None,NaN,NaN,NaN,NaN,NaN
1,2,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,...,741.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN
2,3,2015,1,1,4,US,840,N171US,SFO,CLT,...,811.0,5.0,0,0,None,NaN,NaN,NaN,NaN,NaN
3,4,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,...,756.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN
4,5,2015,1,1,4,AS,135,N527AS,SEA,ANC,...,259.0,-21.0,0,0,None,NaN,NaN,NaN,NaN,NaN
5,6,2015,1,1,4,DL,806,N3730B,SFO,MSP,...,610.0,8.0,0,0,None,NaN,NaN,NaN,NaN,NaN
…,…,…,…,…,…,…,…,…,…,…,...,…,…,…,…,…,…,…,…,…,…
499997,499998,2015,2,3,2,WN,371,N438WN,MDW,PIT,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
499998,499999,2015,2,3,2,WN,1267,N211WN,MDW,TPA,...,927.0,-18.0,0,0,None,NaN,NaN,NaN,NaN,NaN
499999,500000,2015,2,3,2,WN,136,N352SW,MSP,MKE,...,702.0,-23.0,0,0,None,NaN,NaN,NaN,NaN,NaN


In [99]:
run_sql("""
SELECT F.AIRLINE, COUNT(*) AS NumOfFlights
FROM flights F
GROUP BY F.AIRLINE;
""")


,AIRLINE,NumOfFlights
0,AA,46950
1,AS,14149
2,B6,23062
3,DL,68555
4,EV,52965
5,F9,7291
6,HA,6858
7,MQ,31896
8,NK,9324
9,OO,51184


In [100]:
run_sql("""
SELECT F.AIRLINE, AVG(F.DEPARTURE_DELAY) AS Delay
FROM flights F
GROUP BY F.AIRLINE;
""")


,AIRLINE,Delay
0,AA,10.643729
1,AS,3.430815
2,B6,10.774558
3,DL,6.775430
4,EV,10.043026
5,F9,19.664576
6,HA,1.109583
7,MQ,16.141276
8,NK,14.016792
9,OO,12.458124


In [101]:
run_sql("""
SELECT F.AIRLINE, SUM(F.CANCELLED) AS Cancelled
FROM flights F
GROUP BY F.AIRLINE;
""")

,AIRLINE,Cancelled
0,AA,1324
1,AS,83
2,B6,1479
3,DL,938
4,EV,2523
5,F9,122
6,HA,27
7,MQ,3136
8,NK,158
9,OO,1623


In [102]:
run_sql("""
SELECT F.ORIGIN_AIRPORT, F.DESTINATION_AIRPORT, AVG(F.AIR_TIME) AS AVG_FLY_TIME
FROM flights F
GROUP BY F.ORIGIN_AIRPORT, F.DESTINATION_AIRPORT;
""")

,ORIGIN_AIRPORT,DESTINATION_AIRPORT,AVG_FLY_TIME
0,ABE,ATL,111.540541
1,ABE,DTW,83.267606
2,ABE,ORD,112.263158
3,ABI,DFW,32.394191
4,ABQ,ATL,150.263158
5,ABQ,BWI,192.121212
…,…,…,…
4177,YAK,CDV,36.84375
4178,YAK,JNU,35.133333
4179,YUM,PHX,32.188571


In [103]:
run_sql("""
SELECT f1.flight_id AS FLIGHT_ID, f1.ORIGIN_AIRPORT, a1.A_STATE AS ORIGIN_AIRPORT_STATE, a1.AIRPORT AS ORIGIN_AIRPORT_FULL_NAME,
 f1.DESTINATION_AIRPORT, a2.A_STATE as DESTINATION_AIRPORT_STATE, a2.AIRPORT AS DESTINATION_AIRPORT_FULL_NAME
FROM flights f1, airports a1, airports a2
WHERE f1.ORIGIN_AIRPORT = a1.IATA_CODE AND f1.DESTINATION_AIRPORT = a2.IATA_CODE;
""")

,FLIGHT_ID,ORIGIN_AIRPORT,ORIGIN_AIRPORT_STATE,ORIGIN_AIRPORT_FULL_NAME,DESTINATION_AIRPORT,DESTINATION_AIRPORT_STATE,DESTINATION_AIRPORT_FULL_NAME
0,1,ANC,AK,Ted Stevens Anchorage International Airport,SEA,WA,Seattle-Tacoma International Airport
1,2,LAX,CA,Los Angeles International Airport,PBI,FL,Palm Beach International Airport
2,3,SFO,CA,San Francisco International Airport,CLT,NC,Charlotte Douglas International Airport
3,4,LAX,CA,Los Angeles International Airport,MIA,FL,Miami International Airport
4,5,SEA,WA,Seattle-Tacoma International Airport,ANC,AK,Ted Stevens Anchorage International Airport
5,6,SFO,CA,San Francisco International Airport,MSP,MN,Minneapolis-Saint Paul International Airport
…,…,…,…,…,…,…,…
499997,499998,MDW,IL,Chicago Midway International Airport,PIT,PA,Pittsburgh International Airport
499998,499999,MDW,IL,Chicago Midway International Airport,TPA,FL,Tampa International Airport
499999,500000,MSP,MN,Minneapolis-Saint Paul International Airport,MKE,WI,General Mitchell International Airport


In [104]:
run_sql("""
SELECT f.flight_id AS FLIGHT_ID, f.AIRLINE AS IATA_CODE, a.AIRLINE AS AIRLINE_NAME
FROM flights f, airlines a
WHERE f.AIRLINE = a.IATA_CODE;
""")

,FLIGHT_ID,IATA_CODE,AIRLINE_NAME
0,1,AS,Alaska Airlines Inc.
1,2,AA,American Airlines Inc.
2,3,US,US Airways Inc.
3,4,AA,American Airlines Inc.
4,5,AS,Alaska Airlines Inc.
5,6,DL,Delta Air Lines Inc.
…,…,…,…
499997,499998,WN,Southwest Airlines Co.
499998,499999,WN,Southwest Airlines Co.
499999,500000,WN,Southwest Airlines Co.


## Feature Engineering, Data Preprocessing, and Preparation for Modeling

In [105]:
import sqlite3
import pandas as pd

con = sqlite3.connect(db_path)
flights = pd.read_sql_query("SELECT * FROM flights", con)
airports = pd.read_sql_query("SELECT * FROM airports", con)
airlines = pd.read_sql_query("SELECT * FROM airlines", con)
con.commit()
con.close()

In [106]:
flights

,flight_id,A_YEAR,A_MONTH,A_DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,1,2015,1,1,4,AS,98,N407AS,ANC,SEA,...,408.0,-22.0,0,0,None,NaN,NaN,NaN,NaN,NaN
1,2,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,...,741.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN
2,3,2015,1,1,4,US,840,N171US,SFO,CLT,...,811.0,5.0,0,0,None,NaN,NaN,NaN,NaN,NaN
3,4,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,...,756.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN
4,5,2015,1,1,4,AS,135,N527AS,SEA,ANC,...,259.0,-21.0,0,0,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499996,2015,2,3,2,WN,740,N647SW,DAL,MCO,...,910.0,-20.0,0,0,None,NaN,NaN,NaN,NaN,NaN
499996,499997,2015,2,3,2,WN,1711,N8315C,MDW,DEN,...,744.0,-6.0,0,0,None,NaN,NaN,NaN,NaN,NaN
499997,499998,2015,2,3,2,WN,371,N438WN,MDW,PIT,...,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN
499998,499999,2015,2,3,2,WN,1267,N211WN,MDW,TPA,...,927.0,-18.0,0,0,None,NaN,NaN,NaN,NaN,NaN


In [107]:
airports

,IATA_CODE,AIRPORT,CITY,A_STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
...,...,...,...,...,...,...,...
317,WRG,Wrangell Airport,Wrangell,AK,USA,56.48433,-132.36982
318,WYS,Westerly State Airport,West Yellowstone,MT,USA,44.68840,-111.11764
319,XNA,Northwest Arkansas Regional Airport,Fayetteville/Springdale/Rogers,AR,USA,36.28187,-94.30681
320,YAK,Yakutat Airport,Yakutat,AK,USA,59.50336,-139.66023


In [108]:
airlines

,IATA_CODE,AIRLINE
0,UA,United Air Lines Inc.
1,AA,American Airlines Inc.
2,US,US Airways Inc.
3,F9,Frontier Airlines Inc.
4,B6,JetBlue Airways
5,OO,Skywest Airlines Inc.
6,AS,Alaska Airlines Inc.
7,NK,Spirit Air Lines
8,WN,Southwest Airlines Co.
9,DL,Delta Air Lines Inc.


In [109]:
# Adding Latitude and longitude of origin and destination airport:
airp_cols = ["IATA_CODE", "LATITUDE", "LONGITUDE"]
orig = airports[airp_cols].rename(columns=lambda c: "ORIG_" + c if c!="IATA_CODE" else "ORIGIN_AIRPORT")
dest = airports[airp_cols].rename(columns=lambda c: "DEST_" + c if c!="IATA_CODE" else "DESTINATION_AIRPORT")

flights = (flights
           .merge(orig,  on="ORIGIN_AIRPORT",      how="left")
           .merge(dest,  on="DESTINATION_AIRPORT", how="left")
           .merge(airlines, on="AIRLINE",           how="left"))

flights = flights.drop(columns=['IATA_CODE'], errors='ignore')

In [110]:
# Add date column:
flights['FLIGHT_DATE'] = pd.to_datetime(
    flights[['A_YEAR','A_MONTH','A_DAY']].rename(
        columns={'A_YEAR':'year','A_MONTH':'month','A_DAY':'day'}),
    format="%Y-%m-%d"
)

flights['DAY_OF_YEAR'] = flights['FLIGHT_DATE'].dt.dayofyear
flights['WEEK_OF_YEAR'] = flights['FLIGHT_DATE'].dt.isocalendar().week
flights['QUARTER'] = flights['FLIGHT_DATE'].dt.quarter
flights['IS_WEEKEND'] = flights['FLIGHT_DATE'].dt.weekday >= 5  

flights['DEP_HOUR'] = (flights['SCHEDULED_DEPARTURE'] // 100).astype(int)
bins = [0, 6, 12, 18, 24]
labels = ['early_morning','morning','afternoon','evening']
flights['DEP_TIME_BUCKET'] = pd.cut(flights['DEP_HOUR'], bins=bins, labels=labels, right=False)

In [111]:
import pyproj
import numpy as np

# Add distance column:
proj = pyproj.Proj(proj='latlong', datum='WGS84')
def haversine(lat1, lon1, lat2, lon2):
    lat1, lon1, lat2, lon2 = map(np.radians, (lat1, lon1, lat2, lon2))
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dlon/2)**2
    return 2 * 6371 * np.arcsin(np.sqrt(a))  

flights['DISTANCE_KM'] = haversine(
    flights['ORIG_LATITUDE'], flights['ORIG_LONGITUDE'],
    flights['DEST_LATITUDE'], flights['DEST_LONGITUDE']
)

In [112]:
flights = flights.sort_values(["AIRLINE","FLIGHT_DATE"])
flights["ARR_DELAY_FILLED"] = flights["ARRIVAL_DELAY"].fillna(0) 

flights["AIRLINE_7D_MEAN"] = (
    flights
    .groupby("AIRLINE")["ARR_DELAY_FILLED"]
    .rolling(window=7, min_periods=1)
    .mean()
    .reset_index(level=0,drop=True)
)

flights.drop(columns="ARR_DELAY_FILLED", inplace=True)

In [113]:
flights['DELAY_PER_KM'] = flights['ARRIVAL_DELAY'] / flights['DISTANCE_KM']

with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    display(flights)

,flight_id,A_YEAR,A_MONTH,A_DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,DEPARTURE_TIME,DEPARTURE_DELAY,TAXI_OUT,WHEELS_OFF,SCHEDULED_TIME,ELAPSED_TIME,AIR_TIME,DISTANCE,WHEELS_ON,TAXI_IN,SCHEDULED_ARRIVAL,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY,ORIG_LATITUDE,ORIG_LONGITUDE,DEST_LATITUDE,DEST_LONGITUDE,FLIGHT_DATE,DAY_OF_YEAR,WEEK_OF_YEAR,QUARTER,IS_WEEKEND,DEP_HOUR,DEP_TIME_BUCKET,DISTANCE_KM,AIRLINE_7D_MEAN,DELAY_PER_KM
1,2,2015,1,1,4,AA,2336,N3KUAA,LAX,PBI,10,2.0,-8.0,12.0,14.0,280.0,279.0,263.0,2330,737.0,4.0,750,741.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,33.94254,-118.40807,26.68316,-80.09559,2015-01-01,1,1,1,False,0,early_morning,3742.721144,-9.000000,-0.002405
3,4,2015,1,1,4,AA,258,N3HYAA,LAX,MIA,20,15.0,-5.0,15.0,30.0,285.0,281.0,258.0,2342,748.0,8.0,805,756.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN,33.94254,-118.40807,25.79325,-80.29056,2015-01-01,1,1,1,False,0,early_morning,3762.636680,-9.000000,-0.002392
8,9,2015,1,1,4,AA,1112,N3LAAA,SFO,DFW,30,19.0,-11.0,17.0,36.0,195.0,193.0,173.0,1464,529.0,3.0,545,532.0,-13.0,0,0,None,NaN,NaN,NaN,NaN,NaN,37.61900,-122.37484,32.89595,-97.03720,2015-01-01,1,1,1,False,0,early_morning,2352.029788,-10.333333,-0.005527
11,12,2015,1,1,4,AA,1674,N853AA,LAS,MIA,35,27.0,-8.0,21.0,48.0,268.0,266.0,238.0,2174,746.0,7.0,803,753.0,-10.0,0,0,None,NaN,NaN,NaN,NaN,NaN,36.08036,-115.15233,25.79325,-80.29056,2015-01-01,1,1,1,False,0,early_morning,3494.189002,-10.250000,-0.002862
21,22,2015,1,1,4,AA,371,N3GXAA,SEA,MIA,100,52.0,-8.0,30.0,122.0,338.0,347.0,311.0,2724,933.0,6.0,938,939.0,1.0,0,0,None,NaN,NaN,NaN,NaN,NaN,47.44898,-122.30931,25.79325,-80.29056,2015-01-01,1,1,1,False,1,early_morning,4379.482722,-8.000000,0.000228
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,499996,2015,2,3,2,WN,740,N647SW,DAL,MCO,605,558.0,-7.0,8.0,606.0,145.0,132.0,115.0,973,901.0,9.0,930,910.0,-20.0,0,0,None,NaN,NaN,NaN,NaN,NaN,32.84711,-96.85177,28.42889,-81.31603,2015-02-03,34,6,1,False,6,morning,1563.584642,8.428571,-0.012791
499996,499997,2015,2,3,2,WN,1711,N8315C,MDW,DEN,605,605.0,0.0,19.0,624.0,165.0,159.0,133.0,895,737.0,7.0,750,744.0,-6.0,0,0,None,NaN,NaN,NaN,NaN,NaN,41.78598,-87.75242,39.85841,-104.66700,2015-02-03,34,6,1,False,6,morning,1436.930721,7.428571,-0.004176
499997,499998,2015,2,3,2,WN,371,N438WN,MDW,PIT,605,NaN,NaN,NaN,NaN,85.0,NaN,NaN,402,NaN,NaN,830,NaN,NaN,0,1,A,NaN,NaN,NaN,NaN,NaN,41.78598,-87.75242,40.49147,-80.23287,2015-02-03,34,6,1,False,6,morning,645.718100,9.000000,NaN
499998,499999,2015,2,3,2,WN,1267,N211WN,MDW,TPA,605,605.0,0.0,11.0,616.0,160.0,142.0,126.0,997,922.0,5.0,945,927.0,-18.0,0,0,None,NaN,NaN,NaN,NaN,NaN,41.78598,-87.75242,27.97547,-82.53325,2015-02-03,34,6,1,False,6,morning,1606.905349,5.857143,-0.011202


In [114]:
delay_cols = ['AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY']
flights[delay_cols] = flights[delay_cols].fillna(0)

delay_basic_cols = ['DEPARTURE_DELAY', 'ARRIVAL_DELAY']
flights[delay_basic_cols] = flights[delay_basic_cols].fillna(0)

time_cols = ['DEPARTURE_TIME', 'ARRIVAL_TIME', 'WHEELS_ON', 'WHEELS_OFF', 
             'TAXI_OUT', 'TAXI_IN', 'ELAPSED_TIME', 'AIR_TIME']

for col in time_cols:
    flights.loc[flights['CANCELLED'] == 1, col] = flights.loc[flights['CANCELLED'] == 1, col].fillna(-1)

for col in time_cols:
    flights[col] = flights[col].fillna(flights[col].median())

cat_cols = ['AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT']
flights[cat_cols] = flights[cat_cols].fillna('Unknown')

flights = flights.drop(columns=['CANCELLATION_REASON'], errors='ignore')


In [115]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

num_feats = ['DISTANCE_KM', 'AIRLINE_7D_MEAN','DEP_HOUR','DELAY_PER_KM','DEPARTURE_DELAY',
                'ARRIVAL_DELAY', 'TAXI_OUT', 'TAXI_IN', 'AIR_TIME', 'ELAPSED_TIME']

cat_feats = ['DAY_OF_WEEK', 'WEEK_OF_YEAR', 'IS_WEEKEND']
unchanged_columns = ['flight_id', 'AIRLINE', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'FLIGHT_DATE']

num_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='median')),
    ('scale',  StandardScaler())
])

cat_pipeline = Pipeline([
    ('impute', SimpleImputer(strategy='constant', fill_value=-1)),
     ('ordinal', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_feats),
    ('cat', cat_pipeline, cat_feats),
    ('passthrough', 'passthrough', unchanged_columns)
])

X = preprocessor.fit_transform(flights)
y = (flights['ARRIVAL_DELAY'] > 15).astype(int)

In [116]:
drop_cols = ['flight_id','A_YEAR','A_MONTH','A_DAY','SCHEDULED_DEPARTURE',
             'ORIG_LATITUDE','ORIG_LONGITUDE','DEST_LATITUDE','DEST_LONGITUDE', 'TAIL_NUMBER']
flights = flights.drop(columns=drop_cols, errors='ignore')

columns = preprocessor.get_feature_names_out()
processed = pd.DataFrame(X, 
                         index=flights.index,
                         columns = (
                             columns
                         ))
processed['TARGET_LATE'] = y

with pd.option_context('display.max_columns', None, 'display.max_colwidth', None):
    display(processed)

,num__DISTANCE_KM,num__AIRLINE_7D_MEAN,num__DEP_HOUR,num__DELAY_PER_KM,num__DEPARTURE_DELAY,num__ARRIVAL_DELAY,num__TAXI_OUT,num__TAXI_IN,num__AIR_TIME,num__ELAPSED_TIME,cat__DAY_OF_WEEK,cat__WEEK_OF_YEAR,cat__IS_WEEKEND,passthrough__flight_id,passthrough__AIRLINE,passthrough__ORIGIN_AIRPORT,passthrough__DESTINATION_AIRPORT,passthrough__FLIGHT_DATE,TARGET_LATE
1,2.56562,-0.769853,-2.778131,-0.156071,-0.479011,-0.377241,-0.389783,-0.501844,2.103018,1.921332,3.0,0.0,0.0,2,AA,LAX,PBI,2015-01-01,0
3,2.586462,-0.769853,-2.778131,-0.155904,-0.398235,-0.377241,-0.090229,0.119349,2.034866,1.947455,3.0,0.0,0.0,4,AA,LAX,MIA,2015-01-01,0
8,1.110201,-0.83826,-2.778131,-0.196963,-0.559788,-0.477799,0.109474,-0.657142,0.876284,0.798028,3.0,0.0,0.0,9,AA,SFO,DFW,2015-01-01,0
11,2.30552,-0.833984,-2.778131,-0.162059,-0.479011,-0.402381,0.50888,-0.035949,1.762258,1.75153,3.0,0.0,0.0,12,AA,LAS,MIA,2015-01-01,0
21,3.232019,-0.718548,-2.563835,-0.121589,-0.479011,-0.125846,1.407543,-0.191248,2.757276,2.809526,3.0,0.0,0.0,22,AA,SEA,MIA,2015-01-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,0.285058,0.124319,-1.492352,-0.292092,-0.452086,-0.653776,-0.789189,0.274647,0.085722,0.001266,1.0,5.0,0.0,499996,WN,DAL,MCO,2015-02-03,0
499996,0.152509,0.073014,-1.492352,-0.179262,-0.263608,-0.301823,0.309177,-0.035949,0.331069,0.353931,1.0,5.0,0.0,499997,WN,MDW,DEN,2015-02-03,0
499997,-0.675529,0.153636,-1.492352,-0.16338,-0.263608,-0.150986,-1.687852,-1.278336,-1.495401,-1.735937,1.0,5.0,0.0,499998,WN,MDW,PIT,2015-02-03,0
499998,0.330395,-0.007608,-1.492352,-0.271276,-0.263608,-0.603497,-0.489634,-0.346546,0.235656,0.131883,1.0,5.0,0.0,499999,WN,MDW,TPA,2015-02-03,0
